# DBSCAN
데이터가 밀집된 정도(=밀도)를 기준으로 군집을 찾는 알고리즘

In [1]:
import pandas as pd
import folium

In [3]:
# 디스플레이 옵션 설정
pd.set_option('display.width', None) # 출력화면의 너비
pd.set_option('display.max_rows', 100) # 출력할 행의 개수 한도
pd.set_option('display.max_columns',10) # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth', 20) # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True) # 유니코드 사용 너비 조정

# 서울시내 중학교 진학률 데이터셋
file_path = '/content/drive/MyDrive/Colab Notebooks/middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path)

# 열 이름 배열 출력
print(df.columns.values)

['지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고' '외고_국제고'
 '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']


In [4]:
# 지도에 위치 표시
attr = (
    'Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; <a href="https://opentopomap.org">OpenTopoMap</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'
)

tiles = 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'

mschool_map = folium.Map(location=[37.55,126.98], tiles=tiles, attr=attr,
                         zoom_start=12)

# 중학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df['학교명'], df['위도'], df['경도']):
    folium.CircleMarker([lat, lng],
                        radius=5,              # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7,      # 투명도
                        popup=name
    ).add_to(mschool_map)

mschool_map

In [7]:
# 데이터 전처리
# 교유값의 개수
print(df['지역'].nunique())
print(df['코드'].nunique())
print(df['유형'].nunique())
print(df['주야'].nunique())

25
3
3
1


In [8]:
#원핫인코딩 적용
df_encoded = pd.get_dummies(df, columns=['지역','코드','유형','주야'])
df_encoded.head()

,학교명,남학생수,여학생수,일반고,특성화고,...,코드_9,유형_공립,유형_국립,유형_사립,주야_주간
0,서울대학교사범대학부설중학교...,277,0,0.585,0.148,...,False,False,True,False,True
1,서울대학교사범대학부설여자중학교...,0,256,0.680,0.199,...,False,False,True,False,True
2,개원중학교,170,152,0.817,0.047,...,False,True,False,False,True
3,개포중학교,83,72,0.755,0.097,...,False,True,False,False,True
4,경원중학교,199,212,0.669,0.017,...,False,True,False,False,True


# 모형 학습 및 검증

In [9]:
# sklearn 라이브러리에서 cluster 군집 모형 가져오기
from sklearn import cluster
from sklearn import preprocessing

# 분석에 사용할 속성을 선택
train_features = ['과학고', '외고_국제고', '자사고', '자공고',
                  '유형_공립', '유형_국립', '유형_사립',]
X = df_encoded.loc[:, train_features]

# 설명 변수 데이터를 정규화
X = preprocessing.StandardScaler().fit_transform(X)

# DBSCAN 모형 객체 생성
dbm = cluster.DBSCAN(eps=0.2, min_samples=5)

# 모형 학습
dbm.fit(X)

# 예측 (군집)
cluster_label = dbm.labels_
print(cluster_label)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  2 -1 -1 -1  0 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0
 -1 -1  0 -1 -1 -1 -1  1  0 -1 -1  0 -1 -1 -1  2 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1
 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  0 -1 -1 -1  3 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1
  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1  4 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [10]:
# 예측 결과를 데이터프레임에 추가
df_encoded['Cluster'] = cluster_label
df_encoded.head()

,학교명,남학생수,여학생수,일반고,특성화고,...,유형_공립,유형_국립,유형_사립,주야_주간,Cluster
0,서울대학교사범대학부설중학교...,277,0,0.585,0.148,...,False,True,False,True,-1
1,서울대학교사범대학부설여자중학교...,0,256,0.680,0.199,...,False,True,False,True,-1
2,개원중학교,170,152,0.817,0.047,...,True,False,False,True,-1
3,개포중학교,83,72,0.755,0.097,...,True,False,False,True,-1
4,경원중학교,199,212,0.669,0.017,...,True,False,False,True,-1


In [12]:
# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped_cols = ['학교명','과학고','외고_국제고','자사고']
grouped = df_encoded.groupby('Cluster')
for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))
    print(group.loc[:, grouped_cols].head())
    print('\n')

* key : -1
* number : 347
                                학교명  과학고  외고_국제고  자사고
0  서울대학교사범대학부설중학교.....     0.018        0.007   0.227
1  서울대학교사범대학부설여자중학교...   0.000        0.035   0.043
2           개원중학교                   0.009        0.012   0.090
3           개포중학교                   0.013        0.013   0.065
4           경원중학교                   0.007        0.010   0.282


* key : 0
* number : 24
        학교명  과학고  외고_국제고  자사고
47  둔촌중학교     0.0        0.010   0.026
58  성내중학교     0.0        0.013   0.026
62  신명중학교     0.0        0.009   0.031
78  한산중학교     0.0        0.012   0.052
80  강신중학교     0.0        0.012   0.039


* key : 1
* number : 11
             학교명  과학고  외고_국제고  자사고
103      신원중학교     0.0          0.0   0.006
118      개봉중학교     0.0          0.0   0.012
356  서울체육중학교     0.0          0.0   0.000
391    서울광진학교     0.0          0.0   0.000
396    서울정문학교     0.0          0.0   0.000


* key : 2
* number : 5
         학교명  과학고  외고_국제고  자사고
74   천일중학교     0.0        0.003   0.023
1